In [1]:
#!/usr/bin/env bash

<span style="color: red; font-weight: bold">Use the following command to launch a server for this notebook:</span>

```bash
git clone git@github.com:input-output-hk/marlowe-cardano.git
cd marlowe-cardano/marlowe-runtime/
nix run ../marlowe-cli
```

Then navigate to the `examples/` folder in Jupyter and open this notebook.

# Demonstrating the Marlowe Transaction Deposit Component of Marlowe Runtime

## Preliminaries

Record version numbers

In [2]:
marlowe-cli --version

marlowe-cli 0.0.9.0


In [3]:
cardano-cli --version

cardano-cli 1.35.3 - linux-x86_64 - ghc-8.10
git rev 0000000000000000000000000000000000000000


In [4]:
git rev-parse HEAD

595e48a82babf64fab7ffcedb22cda4203b8e751


### Setup the faucet.

Set the location of keys.

In [5]:
TREASURY=treasury

Set the faucet.

In [6]:
FAUCET_SKEY=$TREASURY/payment.skey
FAUCET_ADDR=$(cat $TREASURY/payment.testnet.address)
echo "$FAUCET_ADDR"

addr_test1vq9prvx8ufwutkwxx9cmmuuajaqmjqwujqlp9d8pvg6gupczgtm9j


### Select network

In a separate terminal, set up a tunnel to the Marlowe Runtime development server:
```bash
rm /tmp/preview.socket
ssh -NT \
  -L/tmp/preview.socket:/data/networks/preview/node.socket \
  -L 3717:127.0.0.1:23717 \
  -L 3718:127.0.0.1:23718 \
  -L 3719:127.0.0.1:23719 \
  -L 3721:127.0.0.1:23721 \
  -L 3723:127.0.0.1:23723 \
  54.202.238.5                                              
```

In [7]:
export CARDANO_NODE_SOCKET_PATH=/tmp/preview.socket
export CARDANO_TESTNET_MAGIC=2
MAGIC=(--testnet-magic 2)
echo "${MAGIC[@]}"

--testnet-magic 2


### Check that the reference script has been published

Check that the Marlowe semantics validator was published.

In [8]:
marlowe-cli transaction find-published


Searching for reference script at address: addr_test1vrw0tuh8l95thdqr65dmpcfqnmcw0en7v7vhgegck7gzqgswa07sw

Expected reference script hash: "6a9391d6aa51af28dd876ebb5565b69d1e83e5ac7861506bd29b56b0"

Searching for reference script at address: addr_test1vpa36uuyf95kxpcleldsncedlhjru6vdmh2vnpkdrsz4u6cll9zas

Expected reference script hash: "49076eab20243dc9462511fb98a9cfb719f86e9692288139b7c91df3"
{
    "marlowe": {
        "hash": "6a9391d6aa51af28dd876ebb5565b69d1e83e5ac7861506bd29b56b0",
        "txIn": "087f21f109a997193421a81886ea8c6397d336d19e696457b9c5c7aefdc31873#1"
    },
    "payout": {
        "hash": "49076eab20243dc9462511fb98a9cfb719f86e9692288139b7c91df3",
        "txIn": "087f21f109a997193421a81886ea8c6397d336d19e696457b9c5c7aefdc31873#2"
    }
}


### Participants

#### The Party

In [9]:
PARTY_SKEY="$TREASURY/john-fletcher.skey"
PARTY_VKEY="$TREASURY/john-fletcher.vkey"

Create the first party's keys, if necessary.

In [10]:
if [[ ! -e "$PARTY_SKEY" ]]
then
  cardano-cli address key-gen --signing-key-file "$PARTY_SKEY" --verification-key-file "$PARTY_VKEY"
fi
PARTY_ADDR=$(cardano-cli address build "${MAGIC[@]}" --payment-verification-key-file "$PARTY_VKEY")
echo "$PARTY_ADDR"

addr_test1vqwt2xlr4d8yk4qws675exlqy6pdhq2s76wrehkjggkvr0cerfe8r


Fund the address.

In [11]:
marlowe-cli util fund-address \
  --out-file /dev/null \
  --submit 600 \
  --lovelace 250000000 \
  --source-wallet-credentials "$FAUCET_ADDR:$FAUCET_SKEY" \
  "$PARTY_ADDR"

TxId "94c06da0517dea7089f3ad7b6f131e12af0c20138ebc83b44788a9f20e0e1c25"


#### The Counterparty

In [12]:
COUNTERPARTY_SKEY="$TREASURY/thomas-kyd.skey"
COUNTERPARTY_VKEY="$TREASURY/thomas-kyd.vkey"

Create the second party's keys, if necessary.

In [13]:
if [[ ! -e "$COUNTERPARTY_SKEY" ]]
then
  cardano-cli address key-gen --signing-key-file "$COUNTERPARTY_SKEY" --verification-key-file "$COUNTERPARTY_VKEY"
fi
COUNTERPARTY_ADDR=$(cardano-cli address build "${MAGIC[@]}" --payment-verification-key-file "$COUNTERPARTY_VKEY")
echo "$COUNTERPARTY_ADDR"

addr_test1vr7n0zzth5zycuh972w7rdmh48qur4f3wu6ntn2m2h30dlcvltuy5


Fund the address.

In [14]:
marlowe-cli util fund-address \
  --out-file /dev/null \
  --submit 600  \
  --lovelace 250000000 \
  --source-wallet-credentials "$FAUCET_ADDR:$FAUCET_SKEY" \
  "$COUNTERPARTY_ADDR"

TxId "260c42a1647de8775711f69bb2480c800bea2ced2f4d09c56d1094abdf431b23"


### Time computations

In [15]:
SECOND=1000
MINUTE=$((60 * SECOND))

In [16]:
NOW="$(($(date -u +%s) * SECOND))"
echo "$NOW"

1666209280000


## The Contract

We set the parameters for the ACTUS PAM contract.

In [17]:
MINIMUM_ADA=3000000

FIXED_POINT=1000000
PRINCIPAL=100
INTEREST_RATE=0.02
INTEREST=$(jq -n $PRINCIPAL*$INTEREST_RATE)

STATUS_DATE=$(date -d "$(date -u -R -d @$((NOW/1000)))" +"%Y-%m-%dT00:00:00")
INITIAL_EXCHANGE_DATE=$(date -d "$(date -u -R -d @$((NOW/1000))) + 1 year" +"%Y-01-01T00:00:00")
MATURITY_DATE=$(date -d "$(date -u -R -d @$((NOW/1000))) + 2 year" +"%Y-01-01T00:00:00")

In [18]:
yaml2json << EOI > deposit.actus
scheduleConfig:
  businessDayConvention: "NULL"
  endOfMonthConvention: "EOM"
  calendar: "NC"
maturityDate: "$MATURITY_DATE"
contractId: "0"
enableSettlement: false
initialExchangeDate: "$INITIAL_EXCHANGE_DATE"
contractRole: "RPA"
penaltyType: "O"
cycleAnchorDateOfInterestPayment: "$INITIAL_EXCHANGE_DATE"
contractType: "PAM"
notionalPrincipal: $PRINCIPAL
contractPerformance: "PF"
collateralAmount: 0
dayCountConvention: "30E360"
accruedInterest: 0
statusDate: "$STATUS_DATE"
cycleOfInterestPayment: "P1YL1"
prepaymentEffect: "N"
nominalInterestRate: $INTEREST_RATE
interestCalculationBase: "NT"
EOI
cat deposit.actus

{"accruedInterest":0,"collateralAmount":0,"contractId":"0","contractPerformance":"PF","contractRole":"RPA","contractType":"PAM","cycleAnchorDateOfInterestPayment":"2023-01-01T00:00:00","cycleOfInterestPayment":"P1YL1","dayCountConvention":"30E360","enableSettlement":false,"initialExchangeDate":"2023-01-01T00:00:00","interestCalculationBase":"NT","maturityDate":"2024-01-01T00:00:00","nominalInterestRate":0.02,"notionalPrincipal":100,"penaltyType":"O","prepaymentEffect":"N","scheduleConfig":{"businessDayConvention":"NULL","calendar":"NC","endOfMonthConvention":"EOM"},"statusDate":"2022-10-19T00:00:00"}


Create the contract.

In [19]:
marlowe-cli template actus \
  --minimum-ada "$MINIMUM_ADA" \
  --party "$PARTY_ADDR" \
  --counter-party "$COUNTERPARTY_ADDR" \
  --actus-terms-file  deposit.actus \
  --out-contract-file deposit-1.contract \
  --out-state-file /dev/null

Since we are testing, we don't really want to wait months or years for timeouts, so we edit the contract file to change the maturity date to 15 minutes from now and the initial deposit to 10 minutes from now.

In [20]:
sed -i \
  -e "s/$(jq '.timeout' deposit-1.contract)/$((NOW + 10 * MINUTE))/" \
  -e "s/$(($(date -d "$MATURITY_DATE" -u +%s) * SECOND))/$((NOW + 15 * MINUTE))/" \
  deposit-1.contract

View the contract.

In [21]:
json2yaml deposit-1.contract

timeout: 1666209880000
timeout_continuation: close
when:
- case:
    deposits:
      negate:
        negate: 100000000
    into_account:
      address: addr_test1vqwt2xlr4d8yk4qws675exlqy6pdhq2s76wrehkjggkvr0cerfe8r
    of_token:
      currency_symbol: ''
      token_name: ''
    party:
      address: addr_test1vqwt2xlr4d8yk4qws675exlqy6pdhq2s76wrehkjggkvr0cerfe8r
  then:
    from_account:
      address: addr_test1vqwt2xlr4d8yk4qws675exlqy6pdhq2s76wrehkjggkvr0cerfe8r
    pay: 100000000
    then:
      timeout: 1666210180000
      timeout_continuation: close
      when:
      - case:
          deposits: 2000000
          into_account:
            address: addr_test1vr7n0zzth5zycuh972w7rdmh48qur4f3wu6ntn2m2h30dlcvltuy5
          of_token:
            currency_symbol: ''
            token_name: ''
          party:
            address: addr_test1vr7n0zzth5zycuh972w7rdmh48qur4f3wu6ntn2m2h30dlcvltuy5
        then:
          from_account:
            address: addr_test1vr7n0zzth5zycuh972w7rdm

## Run the Contract

### Transaction 1. Create the contract

Build the transaction.

In [22]:
TX_1=$(
marlowe create \
  --core-file deposit-1.contract \
  --min-utxo "$MINIMUM_ADA" \
  --change-address "$PARTY_ADDR" \
  --address "$PARTY_ADDR" \
  --manual-sign deposit-1.txbody \
| sed -e 's/^.*"\([^\\]*\)\\.*$/\1/' \
)
CONTRACT_ID="$TX_1"
echo "CONTRACT_ID = TX_1 = $CONTRACT_ID"

CONTRACT_ID = TX_1 = a76b3a264f408b294541e566a12c85276669140bed039fc4fe7dfc116a6609a3#1


Sign the transaction.

In [23]:
cardano-cli transaction sign \
  --tx-body-file deposit-1.txbody \
  --out-file     deposit-1.tx \
  --signing-key-file "$PARTY_SKEY"

Submit the transaction using Marlowe Runtime.

In [24]:
marlowe submit deposit-1.tx

"{\"blockHeaderHash\":\"3ca8a3a854add931732d70929eddc4a36c9510bf1793943a5fdef48efb5860a9\",\"blockNo\":286716,\"slotNo\":6206087}"


View the contract's UTxO.

In [25]:
CONTRACT_ADDR=$(marlowe-cli contract address)
echo "$CONTRACT_ADDR"

addr_test1wp4f8ywk4fg672xasahtk4t9k6w3aql943uxz5rt62d4dvqu3c6jv


In [26]:
cardano-cli query utxo "${MAGIC[@]}" --address "$CONTRACT_ADDR" | sed -n -e "1,2p;/${TX_1//#*/}/p"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
a76b3a264f408b294541e566a12c85276669140bed039fc4fe7dfc116a6609a3     1        3000000 lovelace + TxOutDatumHash ScriptDataInBabbageEra "a223586d286fce332859583ecbcdc3c5953606825b3699535ea2306b1cbf1c97"


View the party's UTxOs.

In [27]:
cardano-cli query utxo "${MAGIC[@]}" --address "$PARTY_ADDR"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
a76b3a264f408b294541e566a12c85276669140bed039fc4fe7dfc116a6609a3     0        246785395 lovelace + TxOutDatumNone


View the counterparty's UTxOs.

In [28]:
cardano-cli query utxo "${MAGIC[@]}" --address "$COUNTERPARTY_ADDR"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
260c42a1647de8775711f69bb2480c800bea2ced2f4d09c56d1094abdf431b23     1        250000000 lovelace + TxOutDatumNone


Watch the contract.

In [29]:
marlowe add "$CONTRACT_ID"

a76b3a264f408b294541e566a12c85276669140bed039fc4fe7dfc116a6609a3#1


View the contract's history.

In [30]:
marlowe log --show-contract "$CONTRACT_ID"

transaction a76b3a264f408b294541e566a12c85276669140bed039fc4fe7dfc116a6609a3 (creation)
ContractId:      a76b3a264f408b294541e566a12c85276669140bed039fc4fe7dfc116a6609a3#1
SlotNo:          6206087
BlockNo:         286716
BlockId:         3ca8a3a854add931732d70929eddc4a36c9510bf1793943a5fdef48efb5860a9
ScriptAddress:   addr_test1wp4f8ywk4fg672xasahtk4t9k6w3aql943uxz5rt62d4dvqu3c6jv
Marlowe Version: 1

    When [
      (Case
         (Deposit Address "addr_test1vqwt2xlr4d8yk4qws675exlqy6pdhq2s76wrehkjggkvr0cerfe8r" Address "addr_test1vqwt2xlr4d8yk4qws675exlqy6pdhq2s76wrehkjggkvr0cerfe8r"
            (Token "" "")
            (NegValue
               (NegValue
                  (Constant 100000000))))
         (Pay Address "addr_test1vqwt2xlr4d8yk4qws675exlqy6pdhq2s76wrehkjggkvr0cerfe8r"
            (Party Address "addr_test1vr7n0zzth5zycuh972w7rdmh48qur4f3wu6ntn2m2h30dlcvltuy5")
            (Token "" "")
            (Constant 100000000)
            (When [
               (Case
          

### Transaction 2. Party deposits loan amount

The party deposits the loan amount.

In [31]:
TX_2=$(
marlowe deposit \
  --contract "$CONTRACT_ID" \
  --from-party "$PARTY_ADDR" \
  --to-party "$PARTY_ADDR" \
  --lovelace "$((PRINCIPAL*FIXED_POINT))" \
  --validity-lower-bound "$((NOW-5*MINUTE))" \
  --validity-upper-bound "$((NOW+9*MINUTE))" \
  --change-address "$PARTY_ADDR" \
  --address "$PARTY_ADDR" \
  --manual-sign deposit-2.txbody \
| sed -e 's/^.*"\([^\\]*\)\\.*$/\1/' \
)
echo "TX_2 = $TX_2"

TX_2 = 9c11f0bb01cefebf056dc97532ce55abb966d353897ad3725c785dd3d835d66f


Sign the transaction.

In [32]:
cardano-cli transaction sign \
  --tx-body-file deposit-2.txbody \
  --out-file     deposit-2.tx \
  --signing-key-file "$PARTY_SKEY"

Submit the transaction using Marlowe Runtime.

In [33]:
marlowe submit deposit-2.tx

"{\"blockHeaderHash\":\"ce1f7cd9f9e6058bea751811664fe20257601bccf6bc50c63f04df92fb09a386\",\"blockNo\":286718,\"slotNo\":6206137}"


View the contract's UTxO.

In [34]:
cardano-cli query utxo "${MAGIC[@]}" --address "$CONTRACT_ADDR" | sed -n -e "1,2p;/${TX_2//#*/}/p"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
9c11f0bb01cefebf056dc97532ce55abb966d353897ad3725c785dd3d835d66f     1        3000000 lovelace + TxOutDatumHash ScriptDataInBabbageEra "81f84489c65ccfb55cc8bed3a1faf65b9d57b4234b5f2693e2a0f6047a68d515"


View the party's UTxOs.

In [35]:
cardano-cli query utxo "${MAGIC[@]}" --address "$PARTY_ADDR"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
9c11f0bb01cefebf056dc97532ce55abb966d353897ad3725c785dd3d835d66f     0        146031732 lovelace + TxOutDatumNone


View the counterparty's UTxOs.

In [36]:
cardano-cli query utxo "${MAGIC[@]}" --address "$COUNTERPARTY_ADDR"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
260c42a1647de8775711f69bb2480c800bea2ced2f4d09c56d1094abdf431b23     1        250000000 lovelace + TxOutDatumNone
9c11f0bb01cefebf056dc97532ce55abb966d353897ad3725c785dd3d835d66f     2        100000000 lovelace + TxOutDatumNone


View the contract's history.

In [37]:
marlowe log --show-contract "$CONTRACT_ID"

transaction a76b3a264f408b294541e566a12c85276669140bed039fc4fe7dfc116a6609a3 (creation)
ContractId:      a76b3a264f408b294541e566a12c85276669140bed039fc4fe7dfc116a6609a3#1
SlotNo:          6206087
BlockNo:         286716
BlockId:         3ca8a3a854add931732d70929eddc4a36c9510bf1793943a5fdef48efb5860a9
ScriptAddress:   addr_test1wp4f8ywk4fg672xasahtk4t9k6w3aql943uxz5rt62d4dvqu3c6jv
Marlowe Version: 1

    When [
      (Case
         (Deposit Address "addr_test1vqwt2xlr4d8yk4qws675exlqy6pdhq2s76wrehkjggkvr0cerfe8r" Address "addr_test1vqwt2xlr4d8yk4qws675exlqy6pdhq2s76wrehkjggkvr0cerfe8r"
            (Token "" "")
            (NegValue
               (NegValue
                  (Constant 100000000))))
         (Pay Address "addr_test1vqwt2xlr4d8yk4qws675exlqy6pdhq2s76wrehkjggkvr0cerfe8r"
            (Party Address "addr_test1vr7n0zzth5zycuh972w7rdmh48qur4f3wu6ntn2m2h30dlcvltuy5")
            (Token "" "")
            (Constant 100000000)
            (When [
               (Case
          

### Transaction 3. Counterparty repays the loan's interest

The counterparty repays the interest.

In [38]:
TX_3=$(
marlowe deposit \
  --contract "$CONTRACT_ID" \
  --from-party "$COUNTERPARTY_ADDR" \
  --to-party "$COUNTERPARTY_ADDR" \
  --lovelace "$((INTEREST*FIXED_POINT))" \
  --validity-lower-bound "$((NOW-5*MINUTE))" \
  --validity-upper-bound "$((NOW+9*MINUTE))" \
  --change-address "$COUNTERPARTY_ADDR" \
  --address "$COUNTERPARTY_ADDR" \
  --manual-sign deposit-3.txbody \
| sed -e 's/^.*"\([^\\]*\)\\.*$/\1/' \
)
echo "TX_3 = $TX_3"

TX_3 = b78894ec8135e84f7643f0952d9e557db637d35733dff14c70aae10b37f4bef3


Sign the transaction.

In [39]:
cardano-cli transaction sign \
  --tx-body-file deposit-3.txbody \
  --out-file     deposit-3.tx \
  --signing-key-file "$COUNTERPARTY_SKEY"

Submit the transaction using Marlowe Runtime.

In [40]:
marlowe submit deposit-3.tx

"{\"blockHeaderHash\":\"0256656a718fe660d2bb94a6ed64fc1c792577b813c8c13a2e4bada150c5b198\",\"blockNo\":286719,\"slotNo\":6206160}"


View the contract's UTxO.

In [41]:
cardano-cli query utxo "${MAGIC[@]}" --address "$CONTRACT_ADDR" | sed -n -e "1,2p;/${TX_3//#*/}/p"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
b78894ec8135e84f7643f0952d9e557db637d35733dff14c70aae10b37f4bef3     1        3000000 lovelace + TxOutDatumHash ScriptDataInBabbageEra "08d18019b08d0580cd8240887af6c0483131c345e1a89673cde9a9894e5875ba"


View the party's UTxOs.

In [42]:
cardano-cli query utxo "${MAGIC[@]}" --address "$PARTY_ADDR"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
9c11f0bb01cefebf056dc97532ce55abb966d353897ad3725c785dd3d835d66f     0        146031732 lovelace + TxOutDatumNone
b78894ec8135e84f7643f0952d9e557db637d35733dff14c70aae10b37f4bef3     2        2000000 lovelace + TxOutDatumNone


View the counterparty's UTxOs.

In [43]:
cardano-cli query utxo "${MAGIC[@]}" --address "$COUNTERPARTY_ADDR"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
9c11f0bb01cefebf056dc97532ce55abb966d353897ad3725c785dd3d835d66f     2        100000000 lovelace + TxOutDatumNone
b78894ec8135e84f7643f0952d9e557db637d35733dff14c70aae10b37f4bef3     0        247308089 lovelace + TxOutDatumNone


View the contract's history.

In [44]:
marlowe log --show-contract "$CONTRACT_ID"

transaction a76b3a264f408b294541e566a12c85276669140bed039fc4fe7dfc116a6609a3 (creation)
ContractId:      a76b3a264f408b294541e566a12c85276669140bed039fc4fe7dfc116a6609a3#1
SlotNo:          6206087
BlockNo:         286716
BlockId:         3ca8a3a854add931732d70929eddc4a36c9510bf1793943a5fdef48efb5860a9
ScriptAddress:   addr_test1wp4f8ywk4fg672xasahtk4t9k6w3aql943uxz5rt62d4dvqu3c6jv
Marlowe Version: 1

    When [
      (Case
         (Deposit Address "addr_test1vqwt2xlr4d8yk4qws675exlqy6pdhq2s76wrehkjggkvr0cerfe8r" Address "addr_test1vqwt2xlr4d8yk4qws675exlqy6pdhq2s76wrehkjggkvr0cerfe8r"
            (Token "" "")
            (NegValue
               (NegValue
                  (Constant 100000000))))
         (Pay Address "addr_test1vqwt2xlr4d8yk4qws675exlqy6pdhq2s76wrehkjggkvr0cerfe8r"
            (Party Address "addr_test1vr7n0zzth5zycuh972w7rdmh48qur4f3wu6ntn2m2h30dlcvltuy5")
            (Token "" "")
            (Constant 100000000)
            (When [
               (Case
          

### Transaction 4. Counterparty repays the loan's principal

The counterparty repays the principal.

In [45]:
TX_4=$(
marlowe deposit \
  --contract "$CONTRACT_ID" \
  --from-party "$COUNTERPARTY_ADDR" \
  --to-party "$COUNTERPARTY_ADDR" \
  --lovelace "$((PRINCIPAL*FIXED_POINT))" \
  --validity-lower-bound "$((NOW-5*MINUTE))" \
  --validity-upper-bound "$((NOW+9*MINUTE))" \
  --change-address "$COUNTERPARTY_ADDR" \
  --address "$COUNTERPARTY_ADDR" \
  --manual-sign deposit-4.txbody \
| sed -e 's/^.*"\([^\\]*\)\\.*$/\1/' \
)
echo "TX_4 = $TX_4"

TX_4 = 10ee6b7c8e6828c1ee922432efea72b8bf07de6b37f2b84af6d43b07f6f3681a


Sign the transaction.

In [46]:
cardano-cli transaction sign \
  --tx-body-file deposit-4.txbody \
  --out-file     deposit-4.tx \
  --signing-key-file "$COUNTERPARTY_SKEY"

Submit the transaction using Marlowe Runtime.

In [47]:
marlowe submit deposit-4.tx

"{\"blockHeaderHash\":\"bdbefd99e1559df1cb99a2498b351b80baad8701c3b363458d7da399e4993e8e\",\"blockNo\":286720,\"slotNo\":6206182}"


Check that there is no UTxO for the contract.

In [48]:
cardano-cli query utxo "${MAGIC[@]}" --address "$CONTRACT_ADDR" | sed -n -e "1,2p;/${TX_4//#*/}/p"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------


View the party's UTxOs.

In [49]:
cardano-cli query utxo "${MAGIC[@]}" --address "$PARTY_ADDR"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
10ee6b7c8e6828c1ee922432efea72b8bf07de6b37f2b84af6d43b07f6f3681a     1        103000000 lovelace + TxOutDatumNone
9c11f0bb01cefebf056dc97532ce55abb966d353897ad3725c785dd3d835d66f     0        146031732 lovelace + TxOutDatumNone
b78894ec8135e84f7643f0952d9e557db637d35733dff14c70aae10b37f4bef3     2        2000000 lovelace + TxOutDatumNone


View the counterparty's UTxOs.

In [50]:
cardano-cli query utxo "${MAGIC[@]}" --address "$COUNTERPARTY_ADDR"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
10ee6b7c8e6828c1ee922432efea72b8bf07de6b37f2b84af6d43b07f6f3681a     0        146770238 lovelace + TxOutDatumNone
9c11f0bb01cefebf056dc97532ce55abb966d353897ad3725c785dd3d835d66f     2        100000000 lovelace + TxOutDatumNone


View the contract's history.

In [51]:
marlowe log --show-contract "$CONTRACT_ID"

transaction a76b3a264f408b294541e566a12c85276669140bed039fc4fe7dfc116a6609a3 (creation)
ContractId:      a76b3a264f408b294541e566a12c85276669140bed039fc4fe7dfc116a6609a3#1
SlotNo:          6206087
BlockNo:         286716
BlockId:         3ca8a3a854add931732d70929eddc4a36c9510bf1793943a5fdef48efb5860a9
ScriptAddress:   addr_test1wp4f8ywk4fg672xasahtk4t9k6w3aql943uxz5rt62d4dvqu3c6jv
Marlowe Version: 1

    When [
      (Case
         (Deposit Address "addr_test1vqwt2xlr4d8yk4qws675exlqy6pdhq2s76wrehkjggkvr0cerfe8r" Address "addr_test1vqwt2xlr4d8yk4qws675exlqy6pdhq2s76wrehkjggkvr0cerfe8r"
            (Token "" "")
            (NegValue
               (NegValue
                  (Constant 100000000))))
         (Pay Address "addr_test1vqwt2xlr4d8yk4qws675exlqy6pdhq2s76wrehkjggkvr0cerfe8r"
            (Party Address "addr_test1vr7n0zzth5zycuh972w7rdmh48qur4f3wu6ntn2m2h30dlcvltuy5")
            (Token "" "")
            (Constant 100000000)
            (When [
               (Case
          

## Cleanup

Remove the contract from history tracking.

In [52]:
marlowe rm "$CONTRACT_ID"

a76b3a264f408b294541e566a12c85276669140bed039fc4fe7dfc116a6609a3#1


In [53]:
marlowe ls

Consolidate the UTxOs at the addresses.

In [54]:
marlowe-cli util clean \
  --change-address "$PARTY_ADDR" \
  --required-signer "$PARTY_SKEY" \
  --out-file /dev/null \
  --submit 600

TxId "2982fe67f13eb420ad4be520d2918fd2812b30d488443db33f7b9764dd6ebcbd"


In [55]:
marlowe-cli util clean \
  --change-address "$COUNTERPARTY_ADDR" \
  --required-signer "$COUNTERPARTY_SKEY" \
  --out-file /dev/null \
  --submit 600

TxId "b132a5bca0deec4fa47c8849cbc716e06edc17ed51c17fab3022a9bb77c39c1c"


Send the funds back to the faucet.

In [56]:
marlowe-cli transaction simple \
  --tx-in "$(marlowe-cli util select --lovelace-only 1 "$PARTY_ADDR" | sed -n -e 's/^TxIn "\(.*\)" (TxIx \(.*\))$/\1#\2/;1p')" \
  --tx-in "$(marlowe-cli util select --lovelace-only 1 "$COUNTERPARTY_ADDR" | sed -n -e 's/^TxIn "\(.*\)" (TxIx \(.*\))$/\1#\2/;1p')" \
  --required-signer "$PARTY_SKEY" \
  --required-signer "$COUNTERPARTY_SKEY" \
  --change-address "$FAUCET_ADDR" \
  --out-file /dev/null \
  --submit 600

TxId "56e17c5ec9eec97ca57c8b73399f92363d274834360a3de38f3af4adef71edd6"


See that the funds have been returned to the faucet.

In [57]:
cardano-cli query utxo "${MAGIC[@]}" --address "$PARTY_ADDR" --address "$COUNTERPARTY_ADDR"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
